# Get US universities mail servers

This script uses Python module `dnspython` to gather information on what email providers are used by each American university.

The full list of universities and their web pages was generated from the [National Center for Education Statistics' Integrated Postsecondary Education Data System](https://nces.ed.gov/ipeds/datacenter/Data.aspx). It was saved to the `.data/in/universities_urls.csv` file.

From the same data system, another table was generated containing the estimated enrollments for undergraduate and graduate students in each institution, `data/in/estimated_enrollment.csv`. This information allos to estimate how many students have their data collected and treated by the tech companies.

This work is partially based on Filipe Saraiva's [research](https://gitlab.com/ccsl-ufpa/get-mx-universities/-/blob/master/get-mx-universities.py) for the Federal University of Pará, in Brazil. It is a simpler version, though, focusing solely on the MX servers used. Also, as stated, it uses the module `dnspython` instead of the `host` command, making this script also compatible with Windows.

Further reference:
* https://www.dnspython.org/
* https://gitlab.com/ccsl-ufpa/get-mx-universities/-/blob/master/get-mx-universities.py
* https://stackoverflow.com/questions/4336849/mx-record-lookup-and-check/4339305
* https://www.exclamationsoft.com/exclamationsoft/netmailbot/help/reference/find_mail_server.asp    

In [1]:
import time
import dns.resolver
import pandas as pd
import numpy as np
from collections import Counter
from IPython.display import clear_output
import timeit

## Sets output filename

In [2]:
output_filename = './data/out/output-' + time.strftime('%Y%m%d') + '.csv'

## Imports lists of universities

Imports and cleans files containing the URL list and enrollment data.

### URLs

In [3]:
df_urls = pd.read_csv('./data/in/universities_urls.csv')

# KEEPS ONLY NECESSARY COLUMNS
df_urls = df_urls[['UnitID',
                   'Institution Name',
                   'Institution\'s internet website address (HD2019)']]

# RENAMES COLUMNS
df_urls.columns = ['unit_id', 'name', 'url']

In [4]:
df_urls.head()

,unit_id,name,url
0,177834,A T Still University of Health Sciences,WWW.ATSU.EDU
1,180203,Aaniiih Nakoda College,www.ancollege.edu
2,491464,ABC Adult School,https://www.abcadultschool.edu
3,459523,ABC Beauty Academy,www.abcbeautyacademy.webs.com
4,485500,ABCO Technology,www.abcotechnology.edu


In [36]:
df_urls

,unit_id,name,url
0,177834,A T Still University of Health Sciences,atsu.edu
1,180203,Aaniiih Nakoda College,ancollege.edu
2,491464,ABC Adult School,abcadultschool.edu
3,459523,ABC Beauty Academy,abcbeautyacademy.webs.com
4,485500,ABCO Technology,abcotechnology.edu
...,...,...,...
6522,413820,Yukon Beauty College Inc,yukonbeautycollege.com
6523,451237,Z Hair Academy,zhairacademy.com
6524,204255,Zane State College,zanestate.edu
6525,458575,Zaytuna College,zaytuna.edu


#### Cleans URLS

Througouht the analysis process, a couple of problems were found in the urls data. These are handled below.

In [5]:
# SETS TO LOWER AND STRIPS
df_urls['url'] = df_urls['url'].apply(lambda x: x.lower().strip())

# REMOVE HTTP AND WWW
df_urls['url'] = df_urls['url'].apply(lambda x: x.replace('http://',''))
df_urls['url'] = df_urls['url'].apply(lambda x: x.replace('https://',''))
df_urls['url'] = df_urls['url'].apply(lambda x: x.replace('www.',''))

# REMOVES TRAILING SLASHES
df_urls['url'] = df_urls['url'].apply(lambda x: x.strip('\/'))

# KEEPS ONLY VALUE BEFORE '/' AND '?' 
# for handling cases such as 'xyz.edu/abc/def.html' and 'xyz.edu?utm_campaign=...'
df_urls['url'] = df_urls['url'].apply(lambda x: x.split('/')[0])
df_urls['url'] = df_urls['url'].apply(lambda x: x.split('?')[0])

### Enrollments

In [6]:
df_enrollment = pd.read_csv('./data/in/estimated_enrollment.csv')

# KEEPS ONLY NECESSARY COLUMNS

df_enrollment = df_enrollment[['UnitID',
                               'Institution Name',
                               'Estimated full-time equivalent (FTE) graduate enrollment  2018-19 (EFIA2019)',
                               'Estimated full-time equivalent (FTE) undergraduate enrollment  2018-19 (EFIA2019)']]

# RENAMES COLUMNS
df_enrollment.columns = ['unit_id', 'name', 'est.graduate', 'est.undergrad']

# GENERATES TOTAL NUMBER OF ENROLLMENTS
# FILLING NaN VALUES IS NECESSERY TO ADD BOTH COLUMNS
df_enrollment['est.graduate'] = df_enrollment['est.graduate'].fillna(0)
df_enrollment['est.undergrad'] = df_enrollment['est.undergrad'].fillna(0)
df_enrollment['enrollments'] = df_enrollment['est.graduate'] + df_enrollment['est.undergrad']

In [7]:
df_enrollment.head()

,unit_id,name,est.graduate,est.undergrad,enrollments
0,177834,A T Still University of Health Sciences,1511.0,0.0,1511.0
1,180203,Aaniiih Nakoda College,0.0,125.0,125.0
2,491464,ABC Adult School,0.0,161.0,161.0
3,459523,ABC Beauty Academy,0.0,34.0,34.0
4,485500,ABCO Technology,0.0,54.0,54.0


## Generates full University DF

Joins both DFs into a single one to be used later.

In [8]:
df = df_urls.merge(df_enrollment, on='unit_id', how='left')

df.head()

,unit_id,name_x,url,name_y,est.graduate,est.undergrad,enrollments
0,177834,A T Still University of Health Sciences,atsu.edu,A T Still University of Health Sciences,1511.0,0.0,1511.0
1,180203,Aaniiih Nakoda College,ancollege.edu,Aaniiih Nakoda College,0.0,125.0,125.0
2,491464,ABC Adult School,abcadultschool.edu,ABC Adult School,0.0,161.0,161.0
3,459523,ABC Beauty Academy,abcbeautyacademy.webs.com,ABC Beauty Academy,0.0,34.0,34.0
4,485500,ABCO Technology,abcotechnology.edu,ABCO Technology,0.0,54.0,54.0


## Checking for sanity

In [9]:
# CHECKS FOR DUPLICATED DATA

df[df.duplicated(subset=['url', 'name_y'], keep=False)]

,unit_id,name_x,url,name_y,est.graduate,est.undergrad,enrollments
76,423397,Advantage Technical College,adtecpr.com,Advantage Technical College,0.0,398.0,398.0
77,493071,Advantage Technical College,adtecpr.com,Advantage Technical College,0.0,121.0,121.0
119,486868,All Beauty College,abcollege.edu,All Beauty College,0.0,19.0,19.0
120,459125,All Beauty College,abcollege.edu,All Beauty College,0.0,14.0,14.0
121,485306,All Beauty College,abcollege.edu,All Beauty College,0.0,53.0,53.0
...,...,...,...,...,...,...,...
6227,476896,Wade Gordon Hairdressing Academy,wadegordonacademy.com,Wade Gordon Hairdressing Academy,0.0,88.0,88.0
6358,224679,Western Technical College,westerntech.edu,Western Technical College,0.0,1401.0,1401.0
6359,224660,Western Technical College,westerntech.edu,Western Technical College,0.0,595.0,595.0
6441,481146,Woodruff Medical Training and Testing,woodruffmedical.edu,Woodruff Medical Training and Testing,0.0,289.0,289.0


In [10]:
# CHECKS IF NAME COLUMNS ARE ALWAYS THE SAME

df[df['name_x'] != df['name_y']]

,unit_id,name_x,url,name_y,est.graduate,est.undergrad,enrollments


In [11]:
# KEEPS ONLY ONE NAME COLUMN

df.rename(columns={'name_x' : 'name'},inplace=True)
df.drop('name_y', axis=1, inplace=True)

In [12]:
df.head()

,unit_id,name,url,est.graduate,est.undergrad,enrollments
0,177834,A T Still University of Health Sciences,atsu.edu,1511.0,0.0,1511.0
1,180203,Aaniiih Nakoda College,ancollege.edu,0.0,125.0,125.0
2,491464,ABC Adult School,abcadultschool.edu,0.0,161.0,161.0
3,459523,ABC Beauty Academy,abcbeautyacademy.webs.com,0.0,34.0,34.0
4,485500,ABCO Technology,abcotechnology.edu,0.0,54.0,54.0


Note: some universities share the same name and URL, but not the same ID.

A couple of these cases refer to different units of the same college, and they can be spotted on the full list of URLs before cleaning. These are exception, though.

For this analysis, an conservative approach will be taken for the numbers. Only the records with the highest number of enrollments will be kept in these cases (as opposed to summing the students) and the URLs will be considered only once.

This will affect a mere 108 rows out of 6,527 (1.7%), so it shouldn't impact the outcome of analyses.

In [13]:
df.sort_values('enrollments', ascending=False, inplace=True)
df.drop_duplicates(subset=['name', 'url'], keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)

In [14]:
df

,unit_id,name,url,est.graduate,est.undergrad,enrollments
0,433387,Western Governors University,wgu.edu,26905.0,71587.0,98492.0
1,104717,Grand Canyon University,gcu.edu,31201.0,50730.0,81931.0
2,183026,Southern New Hampshire University,snhu.edu,15496.0,62669.0,78165.0
3,484613,University of Phoenix-Arizona,phoenix.edu,15477.0,52581.0,68058.0
4,232557,Liberty University,liberty.edu,26602.0,40134.0,66736.0
...,...,...,...,...,...,...
6462,124557,University of California-System Administration...,ucop.edu,0.0,0.0,0.0
6463,398130,Mt Sierra College,mtsierra.edu,0.0,0.0,0.0
6464,189264,Beth Hatalmud Rabbinical College,bethhatalmud.com,0.0,0.0,0.0
6465,118189,MTI Business College Inc,mtistockton.com,0.0,0.0,0.0


## Extracts server data

Data will be appended to the `results` list.

Each value in the list will contain a dictionary:
```
{'URL' : the university's URL,
'Response' : list of dicts, in which are stated each server and preference}
```

In [15]:
# SETS THE EMPTY LIST THAT WILL CONTAIN ALL THE RESULTS
results = []

This loop will go through each domain in `hosts_list` and gather two pieces of information:

1. The server with the lowest preference value in the dns.resolve response, labeled `main_server`
1. A full list of the servers in the dns.resolve response, or `server_list`

An `error` boolean value has been added for making life easier when filtering later.

In [16]:
# GENERATES LIST OF URLS FOR ITERATION

hosts_list = df['url'].to_list()

In [17]:
total_runs = 0
start = timeit.default_timer()

for domain in hosts_list:
    clear_output(wait=True)
    total_runs += 1
    try:
        domain_response = []
        simple_server_list = []
        for answer in dns.resolver.resolve(domain, 'MX'):
            answer_data = {'preference' : int(answer.preference),
                          'server' : answer.exchange.to_text()}
            domain_response.append(answer_data)
            simple_server_list.append(answer.exchange.to_text())

        domain_response_sorted = sorted(domain_response, key=lambda k: k['preference'])

        final_data = {'url' : domain,
                     'server_list' : simple_server_list,
                     'main_server' : domain_response_sorted[0]['server'],
                     'error' : False}
    except dns.resolver.NXDOMAIN:
        final_data = {'url' : domain,
                     'server_list' : 'NXDOMAIN',
                     'main_server' : 'NXDOMAIN',
                     'error' : True}
    except dns.resolver.Timeout:
        final_data = {'url' : domain,
                     'server_list' : 'Timeout',
                     'main_server' : 'Timeout',
                     'error' : True} 
    except Exception as e:
        final_data = {'url' : domain,
                     'server_list' : e,
                     'main_server' : e,
                     'error' : True} 
    finally:
        results.append(final_data)
        
    stop = timeit.default_timer()
    
    expected_time = np.round((stop-start) / (total_runs / len(hosts_list)) / 60, 2)
    
    print('WORKING ON URL #', total_runs, 'out of', len(hosts_list), '-', domain)
    print('Progress: ', np.round((total_runs/len(hosts_list) * 100), 2), '%')
    print('Current run time:', np.round((stop - start)/60, 2), 'minutes')
    print('Expected run time:', expected_time, 'minutes')

WORKING ON URL # 6467 out of 6467 - national.edu
Progress:  100.0 %
Current run time: 40.97 minutes
Expected run time: 40.97 minutes


In [18]:
# CHECKS PART OF THE RESULTS
for v in results[:5]:
    print(v, '\n')

{'url': 'wgu.edu', 'server_list': ['us-smtp-inbound-2.mimecast.com.', 'aspmx.l.google.com.', 'alt1.aspmx.l.google.com.', 'alt2.aspmx.l.google.com.', 'aspmx2.googlemail.com.', 'aspmx3.googlemail.com.', 'aspmx4.googlemail.com.', 'aspmx5.googlemail.com.', 'us-smtp-inbound-1.mimecast.com.'], 'main_server': 'us-smtp-inbound-2.mimecast.com.', 'error': False} 

{'url': 'gcu.edu', 'server_list': ['mxa-002bcc03.gslb.pphosted.com.', 'mxb-002bcc03.gslb.pphosted.com.'], 'main_server': 'mxa-002bcc03.gslb.pphosted.com.', 'error': False} 

{'url': 'snhu.edu', 'server_list': ['mxa-0038d801.gslb.pphosted.com.', 'mxb-0038d801.gslb.pphosted.com.'], 'main_server': 'mxa-0038d801.gslb.pphosted.com.', 'error': False} 

{'url': 'phoenix.edu', 'server_list': ['mxa-002dbe01.gslb.pphosted.com.', 'mxb-002dbe01.gslb.pphosted.com.'], 'main_server': 'mxa-002dbe01.gslb.pphosted.com.', 'error': False} 

{'url': 'liberty.edu', 'server_list': ['liberty-edu.mail.protection.outlook.com.'], 'main_server': 'liberty-edu.mail

## Merges extracted data and university info

Turns results into one single Dataframe.

In [19]:
df_results = pd.DataFrame(results)

# SETS ERRORS ASIDE FOR FURTHER INSPECTION
df_errors = df_results[df_results['error']].reset_index(drop=True)
df_results = df_results[~df_results['error']].reset_index(drop=True)

### Checks errors

Inspects the errors to be sure if to changes need to be done in the data extraction.

In [20]:
df_errors

,url,server_list,main_server,error
0,newbrunswick.rutgers.edu,The DNS response does not contain an answer to...,The DNS response does not contain an answer to...,True
1,twin-cities.umn.edu,The DNS response does not contain an answer to...,The DNS response does not contain an answer to...,True
2,www2.gmu.edu,The DNS response does not contain an answer to...,The DNS response does not contain an answer to...,True
3,ucdenver.edu,All nameservers failed to answer the query ucd...,All nameservers failed to answer the query ucd...,True
4,web.uri.edu,The DNS response does not contain an answer to...,The DNS response does not contain an answer to...,True
...,...,...,...,...
526,,The DNS response does not contain an answer to...,The DNS response does not contain an answer to...,True
527,,The DNS response does not contain an answer to...,The DNS response does not contain an answer to...,True
528,learey.edu,The DNS response does not contain an answer to...,The DNS response does not contain an answer to...,True
529,mtsierra.edu,NXDOMAIN,NXDOMAIN,True


In [21]:
errors_list = df_errors['main_server'].to_list()

In [22]:
error_list_clean = []

for v in errors_list:
    error_list_clean.append(str(v)[:25])
    
print(set(error_list_clean))

{'NXDOMAIN', 'Timeout', 'The DNS response does not', 'All nameservers failed to'}


Note: These errors refer to problems in the database and other server errors at the time of the analysis. These will be excluded.

## Creates final DF

In [23]:
df_final = df_results.merge(df, on='url', how='left').drop_duplicates(subset=['url', 'name', 'unit_id'])

## Analysis

### Provider columns

Checks for how many Microsoft or Google servers are in the data. It will look for the following patterns for each provider:

* Google: 'google', 'gmail'
* Microsoft: 'microsoft', 'outlook'

Using the patterns stated above, creates the following columns for the email server providers.

* `main_provider` - classifies the main server provider as Google, Microsoft or Other
* `google` - 1 if Google appears in the list of providers, `server_list`; otherwise, 0
* `microsoft` - 1 if Microsoft appears in the list of providers, `server_list`; otherwise, 0
* `big_tech_clear` - 0 if neither Google nor Microsoft appear in the list of providers, `server_list`; 1 if one of them do; 2 if both

In [24]:
# FOR ADDING NEW PROVIDERS, JUST UPDATE THIS DICT
providers = {
    'Google' : ['google', 'gmail'],
    'Microsoft' : ['microsoft', 'outlook']}

In [25]:
def main_provider_classifier(string, providers=providers):
    '''
    Classifies the main provider, looking for the substrings in the main server URL.
    
    Args:
    string: STR - the string to be checked
    providers: DICT - dictionary containing the providers' names as keys, their patterns as values
    
    Output:
    STR - 'Provider_name' or 'Other', if not in providers
    '''
    for key, value in providers.items():
        if any(substring in string for substring in value):
            return key
    return 'Other'

In [26]:
def check_pattern_in_urls(patterns_to_check, list_of_urls):
    '''
    Checks if a particular pattern from a given list can be found in another list.
    In the context of this particular script, checks if a given provider appears in the dns.resolver response.
    
    Args:
    patterns_to_check: LIST - list of patterns to try to identify in the second list
    list_of_urls: LIST - list of URLs generated from dns.resolver answer object (.exchange value for each row). 
    
    Output:
    1 if any of the pattenrs appear in the list of providers; otherwise, 0
    '''
    for value in patterns_to_check:
        if any(value in url for url in list_of_urls):
            return 1
    return 0

In [27]:
df_final['main_provider'] = df_final['main_server'].apply(main_provider_classifier)
df_final['google'] = df_final['server_list'].apply(lambda x: check_pattern_in_urls(providers['Google'], x))
df_final['microsoft'] = df_final['server_list'].apply(lambda x: check_pattern_in_urls(providers['Microsoft'], x))
df_final['big_tech_clear'] = df_final['google'] + df_final['microsoft']

In [28]:
df_final

,url,server_list,main_server,error,unit_id,name,est.graduate,est.undergrad,enrollments,main_provider,google,microsoft,big_tech_clear
0,wgu.edu,"[us-smtp-inbound-2.mimecast.com., aspmx.l.goog...",us-smtp-inbound-2.mimecast.com.,False,433387,Western Governors University,26905.0,71587.0,98492.0,Other,1,0,1
1,gcu.edu,"[mxa-002bcc03.gslb.pphosted.com., mxb-002bcc03...",mxa-002bcc03.gslb.pphosted.com.,False,104717,Grand Canyon University,31201.0,50730.0,81931.0,Other,0,0,0
2,snhu.edu,"[mxa-0038d801.gslb.pphosted.com., mxb-0038d801...",mxa-0038d801.gslb.pphosted.com.,False,183026,Southern New Hampshire University,15496.0,62669.0,78165.0,Other,0,0,0
3,phoenix.edu,"[mxa-002dbe01.gslb.pphosted.com., mxb-002dbe01...",mxa-002dbe01.gslb.pphosted.com.,False,484613,University of Phoenix-Arizona,15477.0,52581.0,68058.0,Other,0,0,0
4,phoenix.edu,"[mxa-002dbe01.gslb.pphosted.com., mxb-002dbe01...",mxa-002dbe01.gslb.pphosted.com.,False,484631,University of Phoenix-California,879.0,3743.0,4622.0,Other,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28807,hawaii.edu,"[alt2.aspmx.l.google.com., alt1.aspmx.l.google...",aspmx.l.google.com.,False,141963,University of Hawaii System Office,0.0,0.0,0.0,Google,1,0,1
28808,thebeautyschoolofmiddletown.com,[thebeautyschoolofmiddletown-com.mail.protecti...,thebeautyschoolofmiddletown-com.mail.protectio...,False,374927,Beauty School of Middletown,0.0,0.0,0.0,Microsoft,0,1,1
28809,cu.edu,[cu-edu.mail.protection.outlook.com.],cu-edu.mail.protection.outlook.com.,False,128300,University of Colorado System Office,0.0,0.0,0.0,Microsoft,0,1,1
28810,ucop.edu,[ucop-edu.mail.protection.outlook.com.],ucop-edu.mail.protection.outlook.com.,False,124557,University of California-System Administration...,0.0,0.0,0.0,Microsoft,0,1,1


### % that rely on big tech for their main MX server

In [29]:
df_big_tech_percentage = df_final.groupby('main_provider').agg({'url':'count', 'enrollments':'sum'})
df_big_tech_percentage['url_%'] = df_big_tech_percentage['url']/df_final['url'].count()
df_big_tech_percentage['enrollments_%'] = df_big_tech_percentage['enrollments']/df_final['enrollments'].sum()
df_big_tech_percentage

,url,enrollments,url_%,enrollments_%
main_provider,,,,
Google,1070,1721195.0,0.180226,0.110800
Microsoft,2224,6398871.0,0.374600,0.411919
Other,2643,7414235.0,0.445174,0.477282


In [30]:
df_big_tech_percentage['%'] = df_big_tech_percentage.groupby('main_provider')['url'].transform(lambda x: x / x.sum())

### % that rely on big tech for at least one of their MX servers

In [31]:
len(df_final[df_final['big_tech_clear'] > 0])/len(df_final)

0.5662792656223682

No big difference from the ones that have one of these MX as their main servers.

### Universities that use both Google and Microsoft

In [32]:
df_final[df_final['big_tech_clear'] == 2]

,url,server_list,main_server,error,unit_id,name,est.graduate,est.undergrad,enrollments,main_provider,google,microsoft,big_tech_clear
749,triton.edu,"[aspmx.l.google.com., alt1.aspmx.l.google.com....",triton-edu.mail.protection.outlook.com.,False,149532,Triton College,0.0,6695.0,6695.0,Microsoft,1,1,2
771,laverne.edu,"[alt2.aspmx.l.google.com., alt4.aspmx.l.google...",aspmx.l.google.com.,False,117140,University of La Verne,1940.0,4552.0,6492.0,Google,1,1,2
1812,mwcc.edu,"[alt1.aspmx.l.google.com., alt2.aspmx.l.google...",aspmx.l.google.com.,False,166957,Mount Wachusett Community College,0.0,2496.0,2496.0,Google,1,1,2
2570,pierpont.edu,"[pierpont-edu.mail.protection.outlook.com., as...",aspmx.l.google.com.,False,443492,Pierpont Community and Technical College,0.0,1588.0,1588.0,Google,1,1,2
2571,cltcc.edu,"[alt2.aspmx.l.google.com., in.hes.trendmicro.c...",in.hes.trendmicro.com.,False,158088,Central Louisiana Technical Community College,0.0,1588.0,1588.0,Other,1,1,2
2617,amda.edu,"[alt1.aspmx.l.google.com., aspmx2.googlemail.c...",aspmx.l.google.com.,False,188854,American Musical and Dramatic Academy,0.0,1566.0,1566.0,Google,1,1,2
3059,tfc.edu,"[aspmx.l.google.com., alt2.aspmx.l.google.com....",aspmx.l.google.com.,False,141185,Toccoa Falls College,0.0,1224.0,1224.0,Google,1,1,2
6295,arkansasbaptist.edu,[arkansasbaptist-edu.mail.protection.outlook.c...,arkansasbaptist-edu.mail.protection.outlook.com.,False,106306,Arkansas Baptist College,0.0,493.0,493.0,Microsoft,1,1,2
17809,chac.edu,"[alt4.aspmx.l.google.com., chac-edu.mail.prote...",chac-edu.mail.protection.outlook.com.,False,442985,California Healing Arts College,0.0,135.0,135.0,Microsoft,1,1,2
24653,academynursing.com,"[alt4.aspmx.l.google.com., academynursing-com....",academynursing-com.mail.protection.outlook.com.,False,418481,Mifflin County Academy of Science and Technology,0.0,71.0,71.0,Microsoft,1,1,2


### Checking for other providers that might be worth taking a look at

Looks for the most common terms in the "main_server" list to check for particular providers.

For this, instead of counting every single term used in the main_server list, only the one right before '.com' will be considered. This should be enough for spotting any other MX server that appears many times.

In [33]:
# DICT WILL RECEIVE AND COUNT EACH OF THE TERMS

terms = []

In [34]:
for value in df_final['main_server'].to_list():
    # SOME OF THE VALUES END IN '.' SO IT WILL BE NECESSARY TO STRIP THEM
    term_to_extract = value.strip('.').split('.')
    terms.append(term_to_extract[-2].lower())

In [35]:
Counter(terms).most_common(10)

[('outlook', 2222),
 ('google', 1387),
 ('barracudanetworks', 282),
 ('pphosted', 254),
 ('secureserver', 180),
 ('mimecast', 174),
 ('iphmx', 102),
 ('mailspamprotection', 71),
 ('ppe-hosted', 39),
 ('edaff', 39)]